<a href="https://colab.research.google.com/github/MoonJaeHoon/Dacon_Behavioral_Data_Analysis/blob/master/%EB%B2%88%EC%99%B8_%EC%B0%A8%EC%9B%90%EC%B6%95%EC%86%8C_%EC%98%A4%ED%86%A0%EC%9D%B8%EC%BD%94%EB%8D%94%26DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
print( '변경 전 기본 경로 :' )
print(  os.getcwd() )
os.chdir('./drive/My Drive/데이콘_게임데이터분석')
print( '변경 후 파일 경로 :' )
print(  os.getcwd()  )


#import optuna                               # 베이지안 최적화 라이브러리  
#from optuna import Trial
import gc
debug = False
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook              # 진행바
from sklearn.metrics import roc_auc_score   # AUC 스코어 계산
from sklearn.model_selection import KFold   # K-fold CV    
from functools import partial               # 함수 변수 고정
import lightgbm as lgb                      # LightGBM 라이브러리
import warnings                             
warnings.filterwarnings("ignore")           # 경고 문구 미표시
%time
train15 = pd.read_pickle('train15.pkl')

%time
test15 = pd.read_pickle('test15.pkl')

변경 전 기본 경로 :
/content
변경 후 파일 경로 :
/content/drive/My Drive/데이콘_게임데이터분석
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.48 µs
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.72 µs


## Train Set diff생성 (apply 적용)

In [0]:
columns = train15.columns
all_0_index = np.where(train15.apply(lambda x: np.max(x)) == 0)[0].tolist()
wanna_drop_column = columns[all_0_index].drop('P0_Up_Attack_bytime_1')
print(len(wanna_drop_column))

print(train15.shape)
train15 = train15.drop(wanna_drop_column, axis=1)
print(train15.shape)


12
(38872, 561)
(38872, 549)


In [0]:
######################################### 종족 인코딩하기 ########################################
def spec_convert(x):
    if x[0] == 'Z':
        return 2
    elif x[0] == 'P':
        return 1
    else:
        return 0

train15['P0_species']=train15.apply(lambda x: spec_convert(x['P0_species']),axis=1)
train15['P1_species']=train15.apply(lambda x: spec_convert(x['P1_species']),axis=1)

################################ 결측치가 있는 column & 사용 안할 column들 제거해놓기 ######################
no_missing_train = train15.drop(['player0_starting_category','player1_starting_category','player0_center_x','player0_center_y','player1_center_x','player1_center_y',]
                                , axis=1)

################################## 종족이라는 변수는 빼고 P0와 P1간의 차이를 구할 거기 때문에 ##########################
columns = pd.Series(no_missing_train.columns)
columns.index = no_missing_train.columns.tolist()
columns = columns.drop(['P0_species','P1_species','map'])
columns = columns.tolist()

#################### 범주형 변수들 ################# (종족은 순서형으로 봐야될지도 확인ㄱㄱㄱ)
cat_features = ['P0_species','P1_species','map']

####################### P0를 포함하는 컬럼, P1을 포함하는 컬럼 #################################
p0_columns = [c for c in columns if c.startswith('P0_')]
p1_columns = [c for c in columns if c.startswith('P1_')]

####################################### diff변수 생성 전에 루트변환? (카운팅은 높을수록 서로간의 차이가 의미가 없으므로) ##############################
from math import log
from math import sqrt
import re
diff_feat_list = []

for rawfeat0, rawfeat1 in zip(p0_columns,p1_columns):
  #no_missing_train[rawfeat0]  =  no_missing_train.apply(lambda x: sqrt(x[rawfeat0]),axis=1) 
  #no_missing_train[rawfeat1]  =  no_missing_train.apply(lambda x: sqrt(x[rawfeat1]),axis=1) 
  diff_feat = 'diff_'+ re.sub('P0_','', rawfeat0)
  diff_feat_list.append(diff_feat)
  no_missing_train[diff_feat]  =  no_missing_train.apply(lambda x: (x[rawfeat1]-x[rawfeat0]), axis=1)

################################# diff 변수 생성 후 본래 P0, P1 컬럼 없애기 #######################

#no_missing_train = no_missing_train.drop(p0_columns+p1_columns,axis=1)



In [0]:
no_missing_train.describe()


,P0_Ability_bytime_0,P0_Ability_bytime_1,P0_Ability_bytime_2,P0_Ability_bytime_3,P0_Ability_bytime_4,P0_Ability_bytime_5,P0_Ability_bytime_6,P0_Ability_bytime_7,P0_Ability_bytime_8,P0_Ability_bytime_9,P0_Ability_bytime_10,P0_AddToControlGroup_bytime_0,P0_AddToControlGroup_bytime_1,P0_AddToControlGroup_bytime_2,P0_AddToControlGroup_bytime_3,P0_AddToControlGroup_bytime_4,P0_AddToControlGroup_bytime_5,P0_AddToControlGroup_bytime_6,P0_AddToControlGroup_bytime_7,P0_AddToControlGroup_bytime_8,P0_AddToControlGroup_bytime_9,P0_AddToControlGroup_bytime_10,P0_Camera_bytime_0,P0_Camera_bytime_1,P0_Camera_bytime_2,P0_Camera_bytime_3,P0_Camera_bytime_4,P0_Camera_bytime_5,P0_Camera_bytime_6,P0_Camera_bytime_7,P0_Camera_bytime_8,P0_Camera_bytime_9,P0_Camera_bytime_10,P0_ControlGroup_bytime_0,P0_ControlGroup_bytime_1,P0_ControlGroup_bytime_2,P0_ControlGroup_bytime_3,P0_ControlGroup_bytime_4,P0_ControlGroup_bytime_5,P0_ControlGroup_bytime_6,...,diff_Prod_Mp_weight_bytime_4,diff_Prod_Mp_weight_bytime_5,diff_Prod_Mp_weight_bytime_6,diff_Prod_Mp_weight_bytime_7,diff_Prod_Mp_weight_bytime_8,diff_Prod_Mp_weight_bytime_9,diff_Prod_Mp_weight_bytime_10,diff_Patrol_bytime_0,diff_Patrol_bytime_1,diff_Patrol_bytime_2,diff_Patrol_bytime_3,diff_Patrol_bytime_4,diff_Patrol_bytime_5,diff_Patrol_bytime_6,diff_Patrol_bytime_7,diff_Patrol_bytime_8,diff_Patrol_bytime_9,diff_Patrol_bytime_10,diff_Select_Prod_Structure_bytime_0,diff_Select_Prod_Structure_bytime_1,diff_Select_Prod_Structure_bytime_2,diff_Select_Prod_Structure_bytime_3,diff_Select_Prod_Structure_bytime_4,diff_Select_Prod_Structure_bytime_5,diff_Select_Prod_Structure_bytime_6,diff_Select_Prod_Structure_bytime_7,diff_Select_Prod_Structure_bytime_8,diff_Select_Prod_Structure_bytime_9,diff_Select_Prod_Structure_bytime_10,diff_Click_Resources_bytime_0,diff_Click_Resources_bytime_1,diff_Click_Resources_bytime_2,diff_Click_Resources_bytime_3,diff_Click_Resources_bytime_4,diff_Click_Resources_bytime_5,diff_Click_Resources_bytime_6,diff_Click_Resources_bytime_7,diff_Click_Resources_bytime_8,diff_Click_Resources_bytime_9,diff_Click_Resources_bytime_10
count,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,...,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000,38872.000000
mean,3.731066,7.431467,11.961103,18.142236,24.589859,30.575427,36.108922,41.003627,45.159086,48.476178,48.806467,0.276291,0.457502,0.654842,0.881251,1.100689,1.297361,1.481709,1.643934,1.784729,1.904996,1.918450,39.028452,79.440214,127.764509,179.542781,229.039694,273.952794,314.399362,349.860774,380.103879,404.903530,407.357918,0.074347,0.116948,0.154996,0.203437,0.247042,0.286324,0.321594,...,0.009647,0.015345,0.014252,0.022677,0.039772,0.029893,0.038305,-0.002109,-0.001363,-0.001929,0.014303,-0.010728,0.005968,0.015152,0.024671,0.042859,0.055901,0.060764,0.028041,0.068944,0.090554,0.099789,0.136165,0.177583,0.224146,0.238269,0.243003,0.255660,0.249485,-0.052840,-0.096162,-0.137888,-0.160784,-0.159112,-0.177634,-0.179589,-0.173570,-0.168167,-0.168733,-0.167550
std,1.587659,2.544722,3.854273,6.314469,9.633838,13.162384,16.879172,20

## Test Set diff생성 (apply 적용)

In [0]:
test15 = pd.read_pickle('test15.pkl')

print(len(wanna_drop_column))

print(test15.shape)
test15 = test15.drop(wanna_drop_column, axis=1)
print(test15.shape)


12
(16787, 560)
(16787, 548)


In [0]:
######################################### 종족 인코딩하기 ########################################
def spec_convert(x):
    if x[0] == 'Z':
        return 2
    elif x[0] == 'P':
        return 1
    else:
        return 0

test15['P0_species']=test15.apply(lambda x: spec_convert(x['P0_species']),axis=1)
test15['P1_species']=test15.apply(lambda x: spec_convert(x['P1_species']),axis=1)

################################ 결측치가 있는 column & 사용 안할 column들 제거해놓기 ######################
no_missing_test = test15.drop(['player0_starting_category','player1_starting_category','player0_center_x','player0_center_y','player1_center_x','player1_center_y',]
                                , axis=1)

################################## 종족이라는 변수는 빼고 P0와 P1간의 차이를 구할 거기 때문에 ##########################
columns = pd.Series(no_missing_test.columns)
columns.index = no_missing_test.columns.tolist()
columns = columns.drop(['P0_species','P1_species','map'])
columns = columns.tolist()

#################### 범주형 변수들 ################# (종족은 순서형으로 봐야될지도 확인ㄱㄱㄱ)
cat_features = ['P0_species','P1_species','map']

####################### P0를 포함하는 컬럼, P1을 포함하는 컬럼 #################################
p0_columns = [c for c in columns if c.startswith('P0_')]
p1_columns = [c for c in columns if c.startswith('P1_')]

####################################### diff변수 생성 전에 루트변환? (카운팅은 높을수록 서로간의 차이가 의미가 없으므로) ##############################
from math import log
from math import sqrt
import re
diff_feat_list = []

for rawfeat0, rawfeat1 in zip(p0_columns,p1_columns):
  #no_missing_test[rawfeat0]  =  no_missing_test.apply(lambda x: sqrt(x[rawfeat0]),axis=1) 
  #no_missing_test[rawfeat1]  =  no_missing_test.apply(lambda x: sqrt(x[rawfeat1]),axis=1) 
  diff_feat = 'diff_'+ re.sub('P0_','', rawfeat0)
  diff_feat_list.append(diff_feat)
  no_missing_test[diff_feat]  =  no_missing_test.apply(lambda x: (x[rawfeat1]-x[rawfeat0]), axis=1)

################################# diff 변수 생성 후 본래 P0, P1 컬럼 없애기 #######################

#no_missing_test = no_missing_test.drop(p0_columns+p1_columns,axis=1)

no_missing_test.describe()

,P0_Ability_bytime_0,P0_Ability_bytime_1,P0_Ability_bytime_2,P0_Ability_bytime_3,P0_Ability_bytime_4,P0_Ability_bytime_5,P0_Ability_bytime_6,P0_Ability_bytime_7,P0_Ability_bytime_8,P0_Ability_bytime_9,P0_Ability_bytime_10,P0_AddToControlGroup_bytime_0,P0_AddToControlGroup_bytime_1,P0_AddToControlGroup_bytime_2,P0_AddToControlGroup_bytime_3,P0_AddToControlGroup_bytime_4,P0_AddToControlGroup_bytime_5,P0_AddToControlGroup_bytime_6,P0_AddToControlGroup_bytime_7,P0_AddToControlGroup_bytime_8,P0_AddToControlGroup_bytime_9,P0_AddToControlGroup_bytime_10,P0_Camera_bytime_0,P0_Camera_bytime_1,P0_Camera_bytime_2,P0_Camera_bytime_3,P0_Camera_bytime_4,P0_Camera_bytime_5,P0_Camera_bytime_6,P0_Camera_bytime_7,P0_Camera_bytime_8,P0_Camera_bytime_9,P0_Camera_bytime_10,P0_ControlGroup_bytime_0,P0_ControlGroup_bytime_1,P0_ControlGroup_bytime_2,P0_ControlGroup_bytime_3,P0_ControlGroup_bytime_4,P0_ControlGroup_bytime_5,P0_ControlGroup_bytime_6,...,diff_Prod_Mp_weight_bytime_4,diff_Prod_Mp_weight_bytime_5,diff_Prod_Mp_weight_bytime_6,diff_Prod_Mp_weight_bytime_7,diff_Prod_Mp_weight_bytime_8,diff_Prod_Mp_weight_bytime_9,diff_Prod_Mp_weight_bytime_10,diff_Patrol_bytime_0,diff_Patrol_bytime_1,diff_Patrol_bytime_2,diff_Patrol_bytime_3,diff_Patrol_bytime_4,diff_Patrol_bytime_5,diff_Patrol_bytime_6,diff_Patrol_bytime_7,diff_Patrol_bytime_8,diff_Patrol_bytime_9,diff_Patrol_bytime_10,diff_Select_Prod_Structure_bytime_0,diff_Select_Prod_Structure_bytime_1,diff_Select_Prod_Structure_bytime_2,diff_Select_Prod_Structure_bytime_3,diff_Select_Prod_Structure_bytime_4,diff_Select_Prod_Structure_bytime_5,diff_Select_Prod_Structure_bytime_6,diff_Select_Prod_Structure_bytime_7,diff_Select_Prod_Structure_bytime_8,diff_Select_Prod_Structure_bytime_9,diff_Select_Prod_Structure_bytime_10,diff_Click_Resources_bytime_0,diff_Click_Resources_bytime_1,diff_Click_Resources_bytime_2,diff_Click_Resources_bytime_3,diff_Click_Resources_bytime_4,diff_Click_Resources_bytime_5,diff_Click_Resources_bytime_6,diff_Click_Resources_bytime_7,diff_Click_Resources_bytime_8,diff_Click_Resources_bytime_9,diff_Click_Resources_bytime_10
count,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,...,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.00000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000,16787.000000
mean,3.719187,7.397093,11.897957,18.054506,24.435396,30.386251,35.895157,40.770001,44.912015,48.218443,48.555370,0.276524,0.455769,0.652469,0.871806,1.081611,1.268482,1.457378,1.616787,1.753738,1.875499,1.890808,38.897778,78.856794,126.708286,177.903854,226.756002,271.117114,311.207422,346.411688,376.394889,401.124561,403.682909,0.067314,0.106630,0.146125,0.189968,0.223149,0.256806,0.288735,...,0.005004,0.002472,0.011795,0.028474,0.039822,0.079734,0.077679,-0.001549,-0.004825,-0.007685,0.01942,0.034968,0.037410,0.038482,0.042831,0.029308,0.037231,0.042592,0.019539,0.082623,0.110919,0.101090,0.107464,0.054447,0.061893,0.051945,0.085721,0.079585,0.086138,-0.002502,0.013939,0.026687,0.029666,0.026151,0.025555,0.013105,0.025794,0.028951,0.027283,0.029606
std,1.577730,2.508546,3.860135,6.371016,9.698434,13.239429,17.000220,20.869773,24.59

# 변수선택 : Permutation Importance

In [0]:
import lightgbm as lgb
#LGBMC = lgb.LGBMClassifier(n_estimators=100, n_jobs=-1)
#LGBMC.fit(x_train, y_train)
x = no_missing_train.drop(['winner'],axis=1)
y = no_missing_train['winner']

from sklearn.model_selection import train_test_split
tr_X, val_X, tr_y, val_y = train_test_split(x, y,   stratify=y, random_state=0)

## customized function : perm-importance

In [0]:

cat_features = ['P0_species','P1_species','map']
#tr_X = tr_X.drop(not_important_columns, axis=1).copy()
#val_X = val_X.drop(not_important_columns, axis=1).copy()


params = {
    'objective': 'binary',
    #'max_depth': 11,
    'learning_rate': 1e-2, 
    "boosting": "gbdt",#'dart','goss',
    "metric": "auc",
    "verbosity": -1,
    'seed':0,
}
d_train = lgb.Dataset(tr_X, label=tr_y, categorical_feature=cat_features)
d_valid = lgb.Dataset(val_X, label=val_y, categorical_feature=cat_features)
watchlist = [d_train, d_valid]

print('training LGB:')
model = lgb.train(params,
                    train_set=d_train,
                    num_boost_round=10000,
                    valid_sets=watchlist,
                    verbose_eval=500,
                    early_stopping_rounds=750,
                    )

# predictions
#y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration,cat_features=cat_features)


training LGB:
Training until validation scores don't improve for 750 rounds.
[500]	training's auc: 0.792001	valid_1's auc: 0.671564
[1000]	training's auc: 0.863768	valid_1's auc: 0.673913
[1500]	training's auc: 0.911121	valid_1's auc: 0.67434
[2000]	training's auc: 0.942771	valid_1's auc: 0.674131
Early stopping, best iteration is:
[1273]	training's auc: 0.891736	valid_1's auc: 0.674952


In [0]:
from sklearn.metrics import roc_auc_score
metric = roc_auc_score

def permutation_importance(model, X_val, y_val, metric, threshold=0.001,
                           minimize=False, verbose=True):
    results = {}
    
    y_pred = model.predict(X_val, num_iteration=model.best_iteration,cat_features=cat_features)
    
    results['base_score'] = metric(y_val, y_pred)
    if verbose:
        print(f'Base score {results["base_score"]:.5}')
    from tqdm import tqdm_notebook
    for col in tqdm_notebook(X_val.columns):
        freezed_col = X_val[col].copy()

        X_val[col] = np.random.permutation(X_val[col])
        preds = model.predict(X_val, num_iteration=model.best_iteration,cat_features=cat_features)
        results[col] = metric(y_val, preds)

        X_val[col] = freezed_col
        
        if verbose:
            print(f'column: {col} - {results[col]:.5}')
    
    if minimize:
        bad_features = [k for k in results if results[k] < results['base_score'] + threshold]
    else:
        bad_features = [k for k in results if results[k] > results['base_score'] + threshold]
    #bad_features.remove('base_score')
    
    return results, bad_features

bad_features_list = []
mydata  = pd.DataFrame()
seed_list = [0,42,120,565,1000,1357,2014,11515]
for seed in tqdm_notebook(seed_list):
    print('='*20+'Permutation Importance 측정'+'='*20)
    np.random.seed(seed=seed)
    results, bad_features = permutation_importance(model=model,
                                                X_val=val_X,
                                                y_val=val_y,
                                                metric=metric,
                                                minimize=False,
                                                threshold=0.00001,
                                                verbose=False)
    mydata1 = pd.DataFrame(list(results.values()), index=list(results.keys()),columns=['seed_'+str(seed)])
    mydata = pd.concat([mydata,mydata1],axis=1)
    bad_features_list.append(bad_features)
    del mydata1
    del bad_features


====================Permutation Importance 측정====================



====================Permutation Importance 측정====================



====================Permutation Importance 측정====================



====================Permutation Importance 측정====================



====================Permutation Importance 측정====================



====================Permutation Importance 측정====================



====================Permutation Importance 측정====================



====================Permutation Importance 측정====================


In [0]:
print(len(bad_features_list[0]));print(len(bad_features_list[1]));print(len(bad_features_list[2]));print(len(bad_features_list[3]));print(len(bad_features_list[4]))
print(  len(    set(bad_features_list[0]) & set(bad_features_list[1]) & set(bad_features_list[2]) & set(bad_features_list[3])& set(bad_features_list[4])    )   )

245
261
234
244
272
66


In [0]:
mydata['mean_scores'] = mydata.apply(np.mean,axis=1)

In [0]:
mydata.sort_values(by=['mean_scores'], axis=0).iloc[:30]

,seed_0,seed_42,seed_120,seed_565,seed_1000,seed_1357,seed_2014,seed_11515,mean_scores
diff_Ability_bytime_10,0.668005,0.663989,0.663843,0.667407,0.666247,0.665887,0.665158,0.665685,0.665778
time_list,0.668735,0.667796,0.669998,0.668566,0.668403,0.668755,0.668660,0.669162,0.668759
P0_species,0.671656,0.670652,0.670341,0.671362,0.671153,0.671099,0.672122,0.670810,0.671149
diff_Population_bytime_10,0.671374,0.671625,0.670951,0.672836,0.670357,0.672080,0.670118,0.671746,0.671386
diff_Worker_num_bytime_9,0.672115,0.671484,0.672577,0.672507,0.672674,0.673894,0.671478,0.671909,0.672330
diff_Up_Attack_bytime_10,0.673239,0.672858,0.673250,0.673579,0.671795,0.671073,0.673295,0.672365,0.672682
P1_Prod_Mp_weight_bytime_9,0.673537,0.673348,0.673277,0.672548,0.673294,0.673351,0.673503,0.672522,0.673172
diff_Worker_num_bytime_8,0.673134,0.672922,0.673420,0.671955,0.673429,0.673342,0.673828,0.673820,0.673231
P0_Selection_bytime_9,0.673064,0.673177,0.673552,0.673365,0.673767,0.673240,0.672898,0.673310,0.673297
diff_Multi_num_bytime_6,0.673690,0.673566,0.673683,0.673478,0.673472,0.672860,0.673067,0.673318,0.673392


In [0]:
threshold = 1e-5
mean_scores = mydata.apply(np.mean,axis=1)
myindex = np.where(mean_scores>results['base_score'] + threshold
                   )[0]
bad_features = mean_scores.index[myindex].tolist()

print(len(bad_features))

223


In [0]:
import re
not_important_columns = []
for f in bad_features:
    if "P0_" in str(f):
        if re.sub("P0_","P1_", str(f)) in bad_features or re.sub("P0_","diff_", str(f)) not in bad_features :
            not_important_columns.append(f)
    elif "P1_" in str(f):
        if re.sub("P1_","P0_", str(f)) in bad_features or re.sub("P1_","diff_", str(f)) not in bad_features:
            not_important_columns.append(f)
    elif "diff_" in str(f):
        not_important_columns.append(f)
print(  len(not_important_columns)  )
print(not_important_columns)

189
['P0_Ability_bytime_5', 'P0_AddToControlGroup_bytime_0', 'P0_AddToControlGroup_bytime_2', 'P0_Camera_bytime_0', 'P0_Camera_bytime_1', 'P0_Camera_bytime_4', 'P0_Camera_bytime_6', 'P0_Camera_bytime_9', 'P0_GetControlGroup_bytime_1', 'P0_GetControlGroup_bytime_4', 'P0_Right Click_bytime_1', 'P0_Right Click_bytime_3', 'P0_Selection_bytime_0', 'P0_Selection_bytime_1', 'P0_Selection_bytime_2', 'P0_Selection_bytime_4', 'P0_SetControlGroup_bytime_0', 'P0_SetControlGroup_bytime_1', 'P0_SetControlGroup_bytime_2', 'P0_SetControlGroup_bytime_7', 'P0_SetControlGroup_bytime_9', 'P1_Ability_bytime_5', 'P1_Camera_bytime_1', 'P1_Camera_bytime_3', 'P1_ControlGroup_bytime_1', 'P1_ControlGroup_bytime_4', 'P1_GetControlGroup_bytime_4', 'P1_GetControlGroup_bytime_6', 'P1_GetControlGroup_bytime_7', 'P1_GetControlGroup_bytime_9', 'P1_Right Click_bytime_1', 'P1_Right Click_bytime_6', 'P1_Selection_bytime_0', 'P1_Selection_bytime_1', 'P1_Selection_bytime_2', 'P1_Selection_bytime_3', 'P1_Selection_bytime_4',

In [0]:
not_important_columns = ['P0_Ability_bytime_5', 'P0_AddToControlGroup_bytime_0', 'P0_AddToControlGroup_bytime_2', 'P0_Camera_bytime_0', 'P0_Camera_bytime_1', 'P0_Camera_bytime_4', 'P0_Camera_bytime_6', 'P0_Camera_bytime_9', 'P0_GetControlGroup_bytime_1', 'P0_GetControlGroup_bytime_4', 'P0_Right Click_bytime_1', 'P0_Right Click_bytime_3', 'P0_Selection_bytime_0', 'P0_Selection_bytime_1', 'P0_Selection_bytime_2', 'P0_Selection_bytime_4', 'P0_SetControlGroup_bytime_0', 'P0_SetControlGroup_bytime_1', 'P0_SetControlGroup_bytime_2', 'P0_SetControlGroup_bytime_7', 'P0_SetControlGroup_bytime_9', 'P1_Ability_bytime_5', 'P1_Camera_bytime_1', 'P1_Camera_bytime_3', 'P1_ControlGroup_bytime_1', 'P1_ControlGroup_bytime_4', 'P1_GetControlGroup_bytime_4', 'P1_GetControlGroup_bytime_6', 'P1_GetControlGroup_bytime_7', 'P1_GetControlGroup_bytime_9', 'P1_Right Click_bytime_1', 'P1_Right Click_bytime_6', 'P1_Selection_bytime_0', 'P1_Selection_bytime_1', 'P1_Selection_bytime_2', 'P1_Selection_bytime_3', 'P1_Selection_bytime_4', 'P1_SetControlGroup_bytime_0', 'P1_SetControlGroup_bytime_1', 'P1_SetControlGroup_bytime_2', 'P1_SetControlGroup_bytime_6', 'P1_SetControlGroup_bytime_7', 'P1_SetControlGroup_bytime_9', 'P0_Up_Attack_bytime_10', 'P0_Population_bytime_6', 'P0_Gas_num_bytime_2', 'P0_Gas_num_bytime_8', 'P0_Attack_All_bytime_6', 'P0_Attack_All_bytime_10', 'P0_Up_etc_bytime_5', 'P0_Up_etc_bytime_6', 'P0_Up_etc_bytime_7', 'P0_Prod_Mp_bytime_2', 'P0_Build_Tower_bytime_2', 'P0_Build_Tower_bytime_5', 'P0_Patrol_bytime_1', 'P0_Patrol_bytime_3', 'P0_Patrol_bytime_6', 'P0_Patrol_bytime_7', 'P0_Patrol_bytime_9', 'P0_Patrol_bytime_10', 'P0_Select_Prod_Structure_bytime_2', 'P0_Select_Prod_Structure_bytime_6', 'P0_Select_Prod_Structure_bytime_7', 'P0_Select_Prod_Structure_bytime_9', 'P0_Click_Resources_bytime_0', 'P0_Click_Resources_bytime_7', 'P1_Worker_num_bytime_0', 'P1_Worker_num_bytime_2', 'P1_Worker_num_bytime_3', 'P1_Worker_num_bytime_5', 'P1_Up_Armor_bytime_8', 'P1_Population_bytime_6', 'P1_Population_bytime_8', 'P1_Gas_num_bytime_3', 'P1_Gas_num_bytime_8', 'P1_Attack_Worker_bytime_4', 'P1_Attack_Worker_bytime_5', 'P1_Attack_Worker_bytime_7', 'P1_Attack_Worker_bytime_8', 'P1_Attack_Worker_bytime_9', 'P1_Attack_Multi_bytime_2', 'P1_Attack_All_bytime_3', 'P1_Attack_All_bytime_6', 'P1_Attack_All_bytime_8', 'P1_Attack_Tar_bytime_1', 'P1_Attack_Tar_bytime_4', 'P1_Attack_Tar_bytime_8', 'P1_Up_etc_bytime_3', 'P1_Up_etc_bytime_6', 'P1_Up_etc_bytime_7', 'P1_Prod_Mp_bytime_1', 'P1_Prod_Mp_bytime_2', 'P1_Build_Tower_bytime_6', 'P1_Build_Tower_bytime_7', 'P1_Prod_Mp_weight_bytime_4', 'P1_Prod_Mp_weight_bytime_6', 'P1_Prod_Mp_weight_bytime_8', 'P1_Patrol_bytime_1', 'P1_Patrol_bytime_3', 'P1_Patrol_bytime_4', 'P1_Patrol_bytime_7', 'P1_Select_Prod_Structure_bytime_0', 'P1_Select_Prod_Structure_bytime_7', 'P1_Select_Prod_Structure_bytime_9', 'P1_Click_Resources_bytime_0', 'P1_Click_Resources_bytime_1', 'P1_Click_Resources_bytime_3', 'P1_Click_Resources_bytime_8', 'diff_Ability_bytime_5', 'diff_Ability_bytime_6', 'diff_Ability_bytime_7', 'diff_AddToControlGroup_bytime_1', 'diff_AddToControlGroup_bytime_4', 'diff_AddToControlGroup_bytime_6', 'diff_AddToControlGroup_bytime_7', 'diff_AddToControlGroup_bytime_10', 'diff_Camera_bytime_5', 'diff_Camera_bytime_7', 'diff_Camera_bytime_8', 'diff_Camera_bytime_10', 'diff_ControlGroup_bytime_0', 'diff_ControlGroup_bytime_3', 'diff_ControlGroup_bytime_6', 'diff_ControlGroup_bytime_7', 'diff_GetControlGroup_bytime_0', 'diff_GetControlGroup_bytime_2', 'diff_GetControlGroup_bytime_5', 'diff_Right Click_bytime_2', 'diff_Right Click_bytime_4', 'diff_Selection_bytime_0', 'diff_Selection_bytime_1', 'diff_Selection_bytime_7', 'diff_Selection_bytime_8', 'diff_Selection_bytime_9', 'diff_SetControlGroup_bytime_0', 'diff_SetControlGroup_bytime_2', 'diff_SetControlGroup_bytime_4', 'diff_SetControlGroup_bytime_5', 'diff_SetControlGroup_bytime_7', 'diff_SetControlGroup_bytime_9', 'diff_SetControlGroup_bytime_10', 'diff_Worker_num_bytime_4', 'diff_Up_Attack_bytime_6', 'diff_Up_Attack_bytime_7', 'diff_Up_Attack_bytime_8', 'diff_Population_bytime_2', 'diff_Population_bytime_4', 'diff_Population_bytime_6', 'diff_Multi_num_bytime_1', 'diff_Multi_num_bytime_2', 'diff_Gas_num_bytime_0', 'diff_Attack_Worker_bytime_6', 'diff_Attack_Worker_bytime_10', 'diff_Attack_Multi_bytime_7', 'diff_Attack_All_bytime_2', 'diff_Attack_All_bytime_5', 'diff_Attack_All_bytime_7', 'diff_Attack_Tar_bytime_7', 'diff_Attack_Tar_bytime_10', 'diff_Up_etc_bytime_8', 'diff_Prod_Mp_bytime_3', 'diff_Prod_Mp_bytime_7', 'diff_Build_Tower_bytime_1', 'diff_Build_Tower_bytime_4', 'diff_Build_Tower_bytime_9', 'diff_Build_Tower_bytime_10', 'diff_Prod_Mp_weight_bytime_1', 'diff_Prod_Mp_weight_bytime_2', 'diff_Prod_Mp_weight_bytime_5', 'diff_Prod_Mp_weight_bytime_7', 'diff_Patrol_bytime_0', 'diff_Patrol_bytime_2', 'diff_Patrol_bytime_3', 'diff_Patrol_bytime_5', 'diff_Patrol_bytime_7', 'diff_Patrol_bytime_8', 'diff_Select_Prod_Structure_bytime_1', 'diff_Select_Prod_Structure_bytime_3', 'diff_Select_Prod_Structure_bytime_5', 'diff_Select_Prod_Structure_bytime_7', 'diff_Select_Prod_Structure_bytime_8', 'diff_Select_Prod_Structure_bytime_10', 'diff_Click_Resources_bytime_2', 'diff_Click_Resources_bytime_4', 'diff_Click_Resources_bytime_5', 'diff_Click_Resources_bytime_6', 'diff_Click_Resources_bytime_9', 'diff_Click_Resources_bytime_10']
len(not_important_columns)

189

In [0]:
tr_X_reduced = tr_X.drop(not_important_columns, axis=1).copy()
val_X_reduced = val_X.drop(not_important_columns, axis=1).copy()

d_train_reduced = lgb.Dataset(tr_X_reduced, label=tr_y, categorical_feature=cat_features)
d_valid_reduced = lgb.Dataset(val_X_reduced, label=val_y, categorical_feature=cat_features)
watchlist_reduced = [d_train_reduced, d_valid_reduced]

print('training LGB:')
model_reduced = lgb.train(params,
                    train_set=d_train_reduced,
                    num_boost_round=10000,
                    valid_sets=watchlist_reduced,
                    verbose_eval=500,
                    early_stopping_rounds=750,
                    )

y_pred = model_reduced.predict(val_X_reduced)
new_score = metric(val_y, y_pred)

print(f'Original score: {results["base_score"]:.5}, amount of features: {len(results)-1}')
print(f'Score after removing bad_features: {new_score:.5}, amount of features: {tr_X_reduced.shape[1]}')

In [0]:
All_remove_columns = []
All_remove_columns = All_remove_columns + not_important_columns

In [0]:
len(All_remove_columns)

189

# 오토인코더

In [0]:
#not_need_dummies = no_missing_train.drop(['P0_species','P1_species','map','player0_starting','player1_starting','winner'],axis=1)
#need_dummies = no_missing_train.loc[:,['P0_species','P1_species']]
#need_dummies = pd.get_dummies(need_dummies)
#x = pd.concat([not_need_dummies,need_dummies],axis=1)
x = no_missing_train.drop(['winner'],axis=1).drop(All_remove_columns, axis=1)
y = no_missing_train['winner']

not_need_dummies_train = x.drop(cat_features,axis=1) ## scaling & auto encoder 해야 하는 부분
dummies_train = x.loc[:,cat_features]  ## scaling & AE 한 뒤에 concat 해야할 부분
dummies_train = pd.get_dummies(dummies_train.astype('object'))

test_x = no_missing_test.drop(All_remove_columns, axis = 1)
not_need_dummies_test = test_x.drop(cat_features,axis=1) ## scaling & auto encoder 해야 하는 부분
dummies_test = test_x.loc[:,cat_features] ## scaling & AE 한 뒤에 concat 해야할 부분
dummies_test = pd.get_dummies(dummies_test.astype('object'))

## 200차원으로 축소

## 150차원으로 축소

In [0]:
from sklearn.preprocessing import StandardScaler
from keras.layers import Input
scaler = StandardScaler()
scaler.fit(not_need_dummies_train)
train_x = scaler.transform(not_need_dummies_train)
test_x = scaler.transform(not_need_dummies_test)

# define the number of features
ncol = train_x.shape[1]
print(ncol)
### Define the encoder dimension
encoding_dim = 150
input_dim = Input(shape = (ncol, ))

619


In [0]:
#케라스를 통해 모델 생성을 시작합니다.

import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Dropout , Lambda, Flatten, LeakyReLU, ELU
from keras.optimizers import Adam, SGD,RMSprop
from keras import  backend as K
from keras import metrics
from keras.layers import BatchNormalization # 배치정규화
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.utils.generic_utils import get_custom_objects
################ Gelu #############
class Gelu(Activation):
    def __init__(self, activation, **kwargs):
        super(Gelu, self).__init__(activation, **kwargs)
        self.__name__='gelu'
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

get_custom_objects().update({'gelu': Activation(gelu)})


# Encoder Layers
encoded1 = Dense(594, activation='elu')(input_dim)
#encoded1 = Dropout(0.15)(encoded1)
encoded2 = Dense(569, activation =  'elu')(encoded1)
encoded3 = Dense(544, activation =  'elu')(encoded2)
encoded4 = Dense(519, activation =  'elu')(encoded3)
encoded5 = Dense(494, activation =  'elu')(encoded4)
encoded6 = Dense(469, activation =  'elu')(encoded5)
encoded7 = Dense(444, activation =  'elu')(encoded6)
encoded8 = Dense(419, activation =  'elu')(encoded7)
encoded9 = Dense(394, activation =  'elu')(encoded8)
encoded10 = Dense(369, activation =  'elu')(encoded9)
encoded11 = Dense(344, activation =  'elu')(encoded10)
encoded12 = Dense(319, activation =  'elu')(encoded11)
encoded13 = Dense(294, activation =  'elu')(encoded12)
encoded14 = Dense(269, activation =  'elu')(encoded13)
encoded15 = Dense(244, activation =  'elu')(encoded14)
encoded16 = Dense(219, activation =  'elu')(encoded15)
encoded17 = Dense(194, activation =  'elu')(encoded16)
encoded18 = Dense(169, activation =  'elu')(encoded17)
encoded19 = Dense(encoding_dim, activation =  'elu')(encoded18)

# Decoder Layers
decoded1 = Dense(169, activation =  'elu')(encoded19)
decoded2 = Dense(194, activation =  'elu')(decoded1)
decoded3 = Dense(219, activation =  'elu')(decoded2)
decoded4 = Dense(244, activation =  'elu')(decoded3)
decoded5 = Dense(269, activation =  'elu')(decoded4)
decoded6 = Dense(294, activation =  'elu')(decoded5)
decoded7 = Dense(319, activation =  'elu')(decoded6)
decoded8 = Dense(344, activation = 'elu')(decoded7)
decoded9 = Dense(369, activation = 'elu')(decoded8)
decoded10 = Dense(394, activation = 'elu')(decoded9)
decoded11 = Dense(419, activation = 'elu')(decoded10)
decoded12 = Dense(444, activation = 'elu')(decoded11)
decoded13 = Dense(469, activation = 'elu')(decoded12)
decoded14 = Dense(494, activation = 'elu')(decoded13)
decoded15 = Dense(519, activation = 'elu')(decoded14)
decoded16 = Dense(544, activation = 'elu')(decoded15)
decoded17 = Dense(569, activation = 'elu')(decoded16)
decoded18 = Dense(594, activation = 'elu')(decoded17)
decoded19 = Dense(ncol, activation = 'linear')(decoded18)

# Combine Encoder and Deocder layers
autoencoder = Model(inputs = input_dim, outputs = decoded19)
#autoencoder.summary()


pat =  20
red_pat =  5
initial_rate = 1e-4
factor = 1/2 ## 1/np.sqrt(10) # red_patience 만큼 기다리다가, 학습률을 *factor 배로 줄여버림 
minimumlr = 1e-10

# Compile the Model
autoencoder.compile(
                    optimizer=Adam(lr=initial_rate),
                    loss = 'mae'
                    ,metrics=['mse']
                    )

rlr = ReduceLROnPlateau(monitor='val_loss', factor = factor,   # patience 만큼 기다리다가 0.1이면 학습률을 0.1배로 줄여버림 
                        patience = red_pat, mode = 'min', verbose = 1,
                        min_lr = minimumlr
                        )
es = EarlyStopping(monitor= 'val_loss', patience = pat, verbose = 1, mode='min',
                    restore_best_weights = True
                   ) 

In [0]:
# reduce factor = 1/2
from sklearn.model_selection import train_test_split
X_train, X_valid = train_test_split(train_x, test_size=0.2, random_state=0)

# Train Auto Encoder
autoencoder.fit(X_train, X_train, epochs = 500, batch_size = 128, validation_data=(X_valid,X_valid),callbacks=[rlr   ,es
                                                                                                              ]  )

Train on 31097 samples, validate on 7775 samples
Epoch 1/500
31097/31097 [==============================] - 14s 458us/step - loss: 0.4708 - mean_squared_error: 0.8112 - val_loss: 0.4155 - val_mean_squared_error: 0.7037
Epoch 2/500
31097/31097 [==============================] - 5s 152us/step - loss: 0.3888 - mean_squared_error: 0.6476 - val_loss: 0.3716 - val_mean_squared_error: 0.6240
Epoch 3/500
31097/31097 [==============================] - 4s 144us/step - loss: 0.3609 - mean_squared_error: 0.6052 - val_loss: 0.3511 - val_mean_squared_error: 0.5978
Epoch 4/500
31097/31097 [==============================] - 5s 152us/step - loss: 0.3432 - mean_squared_error: 0.5805 - val_loss: 0.3361 - val_mean_squared_error: 0.5748
Epoch 5/500
31097/31097 [==============================] - 5s 151us/step - loss: 0.3291 - mean_squared_error: 0.5607 - val_loss: 0.3255 - val_mean_squared_error: 0.5637
Epoch 6/500
31097/31097 [==============================] - 5s 148us/step - loss: 0.3215 - mean_squared_er

In [0]:

############### 오토인코더 모델 불러오기 #############
encoder = Model(inputs = input_dim, outputs = encoded19)
encoded_input = Input(shape = (encoding_dim, ))

encoded_train = pd.DataFrame(encoder.predict(train_x))
encoded_train = encoded_train.add_prefix('feature_')
#dummies_train = pd.get_dummies(dummies_train.astype('object'))
encoded_train = pd.concat([encoded_train,dummies_train],axis=1)

encoded_test = pd.DataFrame(encoder.predict(test_x))
encoded_test = encoded_test.add_prefix('feature_')
#dummies_test = pd.get_dummies(dummies_test.astype('object'))
encoded_test = pd.concat([encoded_test,dummies_test],axis=1)

print(encoded_train.shape)
print(encoded_test.shape)

encoded_train.to_pickle('train_encoded.pkl')
encoded_test.to_pickle('test_encoded.pkl')

encoded_train.head()
#encoded_test.head()


(38872, 163)
(33574, 163)


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,P0_species_0,P0_species_1,P0_species_2,P1_species_0,P1_species_1,P1_species_2,map_0.0,map_1.0,map_2.0,map_3.0,map_4.0,map_5.0,map_6.0
0,0.006039,0.048173,0.029733,0.009024,-0.019873,0.050236,-0.023526,0.043908,-0.014531,0.021740,-0.038355,0.018858,0.007701,-0.057238,-0.037549,0.028151,0.003831,-0.028500,0.010079,0.042021,0.035821,-0.010235,0.004585,0.019037,-0.013382,0.060005,-0.033618,-0.054798,0.014237,0.026817,-0.016972,0.008659,-0.036176,0.019435,-0.013160,-0.027967,-0.008257,0.060012,-0.055919,0.006245,...,-0.046500,-0.016597,-0.043483,0.001863,0.023887,0.015872,-0.001798,0.036447,-0.015658,0.007362,0.016685,-0.005421,-0.017038,-0.009105,-0.015562,-0.011924,-0.062659,0.003879,-0.053845,0.018290,-0.048927,-0.084534,-0.037699,-0.076666,-0.052653,0.017865,-0.012118,1,0,0,1,0,0,0,0,0,0,1,0,0
1,0.055885,0.018658,0.110217,-0.069923,-0.028489,-0.042918,0.019199,0.034121,-0.088668,0.040965,-0.045495,0.031146,-0.113946,-0.077115,-0.038872,0.029399,-0.000867,0.043051,0.033734,-0.007818,-0.042923,-0.061943,-0.055774,-0.063537,-0.025412,-0.062436,-0.037730,0.013633,0.008218,0.019985,0.002333,0.079368,0.079351,-0.003467,-0.023424,0.009982,0.008121,0.072396,-0.045198,0.025088,...,-0.021356,-0.011303,0.038751,0.023645,0.028017,0.050664,0.009834,0.041341,-0.043812,-0.015909,-0.007906,0.013474,-0.053851,-0.049162,-0.039062,0.031125,0.020927,-0.008624,0.001464,0.041049,0.098142,-0.094166,0.020727,-0.033473,-0.047493,0.060276,0.030214,0,1,0,1,0,0,0,0,1,0,0,0,0
2,0.019572,-0.067619,0.091963,-0.037260,0.094452,-0.029897,-0.004285,0.014527,0.006834,-0.052115,-0.019086,0.077539,-0.076259,0.002464,0.023236,0.044872,-0.011157,0.063080,-0.074400,-0.060838,0.009842,0.023343,-0.097727,0.011705,0.041338,0.034140,0.027098,-0.005210,0.029896,-0.013275,-0.029593,0.018764,0.016921,0.054359,-0.029281,0.076100,0.089780,0.077119,-0.035511,-0.038343,...,-0.020958,0.014628,-0.016348,-0.032883,0.051480,0.021776,0.037176,-0.014885,-0.059191,0.009547,0.035283,0.035823,0.087417,0.018617,0.026684,0.026028,-0.019102,-0.033812,0.019102,0.070545,0.048364,-0.048170,0.086405,0.056267,-0.013105,0.122748,0.083448,0,1,0,0,0,1,1,0,0,0,0,0,0
3,-0.063112,-0.119241,0.001095,-0.030070,-0.045951,-0.010644,0.055667,0.025599,-0.022881,0.045441,-0.032410,0.080988,-0.036489,0.105250,0.021737,0.022567,-0.101853,-0.016435,0.054014,-0.075366,-0.042652,0.053001,-0.001926,0.064528,-0.064901,0.112971,0.023500,0.075860,0.061597,0.015274,0.004002,-0.016144,0.066573,-0.011186,0.058048,0.004241,0.032195,0.013567,0.025113,0.105102,...,0.128505,-0.058465,-0.055645,-0.039861,0.038839,-0.013649,0.019766,-0.019121,0.044311,-0.002709,-0.038017,-0.065324,-0.002504,0.007143,0.031992,-0.091746,0.064197,0.040817,-0.043442,-0.051621,-0.045577,0.064417,-0.043148,0.075256,0.015099,-0.061543,0.004729,1,0,0,0,1,0,0,0,0,0,0,0,1
4,0.018595,-0.008071,0.008608,0.098952,0.012093,-0.000996,0.005812,-0.058995,-0.048174,-0.032681,-0.074533,-0.053968,-0.062718,-0.013241,-0.046849,0.031147,-0.027376,0.014943,-0.003041,0.030131,0.074259,-0.026970,-0.079229,-0.084359,0.011840,0.045498,0.029474,-0.016773,-0.059369,-0.061698,0.018372,0.009297,-0.013388,0.056303,0.024029,-0.015526,-0.038057,0.010557,-0.040114,-0.025502,...,0.014906,-0.018048,0.010670,0.008242,-0.002332,-0.0162

In [0]:
from sklearn.model_selection import train_test_split
train_input, cv_input, train_target, cv_target = train_test_split(encoded_train, y ,stratify=y, test_size=0.2, random_state=0)

In [0]:
import numpy as np
import pandas as pd

#random seeds for stochastic parts of neural network 
np.random.seed(123)
from tensorflow import set_random_seed
set_random_seed(123)

from keras.models import Sequential, Model, load_model
from keras.layers import  Activation,  Lambda, Flatten, LeakyReLU, ELU, Dense
from keras.layers import Input, Concatenate, Reshape, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding
from sklearn.model_selection import StratifiedKFold
from keras.optimizers import Adam, SGD,RMSprop
from keras import  backend as K
from keras import metrics
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten
#from swa.keras import SWA # swa optimizer - https://pypi.org/project/keras-swa/
import tensorflow as tf
from keras.utils.generic_utils import get_custom_objects
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

################ Gelu #############

class Gelu(Activation):
    def __init__(self, activation, **kwargs):
        super(Gelu, self).__init__(activation, **kwargs)
        self.__name__='gelu'
        
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

get_custom_objects().update({'gelu': Activation(gelu)})


def create_nn_model():
    inp = Input(shape=(163,))
    x = Dense(512)(inp)
    x = BatchNormalization()(x)
    x = ELU()(x)
#    x = Activation(gelu)(x)
#    x = Dropout(rate = 0.4)(x)
    
    x = Dense(510)(x)
    x = BatchNormalization()(x)
    x = ELU()(x)
#    x = Activation(gelu)(x)
#    x = Dropout(rate = 0.4)(x)

    x = Dense(260)(x)
    x = BatchNormalization()(x)
    x = ELU()(x)
#    x = Activation(gelu)(x)
    x = Dropout(rate = 0.15)(x)

    x = Dense(258)(x)
    x = BatchNormalization()(x)
    x = ELU()(x)
#    x = Activation(gelu)(x)
    x = Dropout(rate = 0.15)(x)

    x = Dense(132)(x)
    x = BatchNormalization()(x)
    x = ELU()(x)
#    x = Activation(gelu)(x)
    x = Dropout(rate = 0.1)(x)

    x = Dense(130)(x)
#    x = BatchNormalization()(x)
    x = ELU()(x)
#    x = Activation(gelu)(x)
#    x = Dropout(rate = 0.25)(x)


    out = Dense(2, activation='softmax')(x) #scalar_coupling_constant    
    model = Model(inputs=inp, outputs=out)
    return model

nn_model = create_nn_model(  )

In [0]:
epoch =  800
pat =  50
red_pat =  5
batchsize = 128
initial_rate = 1e-4
factor = 1/np.sqrt(10) # red_patience 만큼 기다리다가, 학습률을 *factor 배로 줄여버림 
minimumlr = 1e-7

### model early stopping : 더이상 성능 개선이 되지 않으면 멈춤
es = EarlyStopping(monitor= 'val_loss', patience = pat, verbose = 1, mode='auto',
                    restore_best_weights = True
                   ) 

### model check point
#mc = ModelCheckpoint(filepath=check_path, monitor='val_loss', mode='auto', save_best_only=True)

## ReduceLR on Plateau : val_loss가 안 줄어들 때 lr을 작게 할 수 있음 (local minima 대처방법)
rlr = ReduceLROnPlateau(monitor='val_loss', factor = factor,   # patience 만큼 기다리다가 0.1이면 학습률을 0.1배로 줄여버림 
                        patience = red_pat, mode = 'auto', verbose = 1,
                        min_lr = minimumlr
                        )

from keras import optimizers
optimizer = optimizers.Adam(
    lr=initial_rate,
)

## compile model
import tensorflow as tf
#metrics = tf.keras.metrics.AUC
#metrics = tf.contrib.metrics.streaming_auc
metrics = 'accuracy'
nn_model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=[metrics]
              )

## fitting model
hist = nn_model.fit(  train_input, train_target,validation_data=[cv_input, cv_target],
                    batch_size=batchsize,
                    epochs=epoch,
                    callbacks = [es 
                                 #,mc
                                 ,rlr
                                ] )

Train on 31097 samples, validate on 7775 samples
Epoch 1/800
31097/31097 [==============================] - 16s 518us/step - loss: 0.7099 - acc: 0.5604 - val_loss: 0.6656 - val_acc: 0.5911
Epoch 2/800
31097/31097 [==============================] - 4s 127us/step - loss: 0.6695 - acc: 0.5902 - val_loss: 0.6650 - val_acc: 0.5981
Epoch 3/800
31097/31097 [==============================] - 4s 131us/step - loss: 0.6604 - acc: 0.6003 - val_loss: 0.6579 - val_acc: 0.6060
Epoch 4/800
31097/31097 [==============================] - 4s 129us/step - loss: 0.6549 - acc: 0.6110 - val_loss: 0.6539 - val_acc: 0.6109
Epoch 5/800
31097/31097 [==============================] - 4s 125us/step - loss: 0.6501 - acc: 0.6116 - val_loss: 0.6522 - val_acc: 0.6114
Epoch 6/800
31097/31097 [==============================] - 4s 128us/step - loss: 0.6457 - acc: 0.6180 - val_loss: 0.6491 - val_acc: 0.6219
Epoch 7/800
31097/31097 [==============================] - 4s 128us/step - loss: 0.6417 - acc: 0.6270 - val_loss: 0.